# Data Preprocessing - Bank Loan Analytics

**Purpose:** Clean and prepare the loan data for analysis.

**Approach:** Document every decision with business justification.

In [5]:
# Import Libraries
import pandas as pd
import numpy as np

# Display Settings for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 1. Load Data and Initial Inspection 

In [6]:
# Loading raw data
df = pd.read_csv('../data/raw/loan_data.csv')

print(f'Dataset Shape: {df.shape[0]} rows, {df.shape[1]} columns')

Dataset Shape: 45000 rows, 14 columns


In [7]:
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22,female,Master,71948,0,RENT,35000,PERSONAL,16.02,0.49,3,561,No,1
1,21,female,High School,12282,0,OWN,1000,EDUCATION,11.14,0.08,2,504,Yes,0
2,25,female,High School,12438,3,MORTGAGE,5500,MEDICAL,12.87,0.44,3,635,No,1
3,23,female,Bachelor,79753,0,RENT,35000,MEDICAL,15.23,0.44,2,675,No,1
4,24,male,Master,66135,1,RENT,35000,MEDICAL,14.27,0.53,4,586,No,1


## 2. Data Type Assessment

**Why this matters:**
- Wrong data types can cause calculation errors.
- Categorical columns need to be identified.
- Helps us understand what kind of analysis is possible.

In [8]:
#Checking data types for each column
df.dtypes

person_age                          int64
person_gender                         str
person_education                      str
person_income                       int64
person_emp_exp                      int64
person_home_ownership                 str
loan_amnt                           int64
loan_intent                           str
loan_int_rate                     float64
loan_percent_income               float64
cb_person_cred_hist_length          int64
credit_score                        int64
previous_loan_defaults_on_file        str
loan_status                         int64
dtype: object

In [7]:
# Check for missing values
print('---Missing Values---')
print(df.isnull().sum())

---Missing Values---
person_age                        0
person_gender                     0
person_education                  0
person_income                     0
person_emp_exp                    0
person_home_ownership             0
loan_amnt                         0
loan_intent                       0
loan_int_rate                     0
loan_percent_income               0
cb_person_cred_hist_length        0
credit_score                      0
previous_loan_defaults_on_file    0
loan_status                       0
dtype: int64


In [9]:
# Checking for unique values in categorical cols
categorical_cols = ['person_gender','person_education','person_home_ownership','loan_intent','previous_loan_defaults_on_file']

print('---Unique Values in Categorical Columns---')
for col in categorical_cols:
    print(f'\n{col}:')
    print(f" Values: {df[col].unique().tolist()}")
    print(f' Count: {df[col].nunique()} unique values')

---Unique Values in Categorical Columns---

person_gender:
 Values: ['female', 'male']
 Count: 2 unique values

person_education:
 Values: ['Master', 'High School', 'Bachelor', 'Associate', 'Doctorate']
 Count: 5 unique values

person_home_ownership:
 Values: ['RENT', 'OWN', 'MORTGAGE', 'OTHER']
 Count: 4 unique values

loan_intent:
 Values: ['PERSONAL', 'EDUCATION', 'MEDICAL', 'VENTURE', 'HOMEIMPROVEMENT', 'DEBTCONSOLIDATION']
 Count: 6 unique values

previous_loan_defaults_on_file:
 Values: ['No', 'Yes']
 Count: 2 unique values


## 3. Statistical Summary - Spotting Outliers

Looking at min, max, and mean to identify suspicious values.

In [10]:
# Statistical Summary of Numerical Columns
df.describe()

,person_age,person_income,person_emp_exp,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,loan_status
count,45000.000000,4.500000e+04,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000
mean,27.764178,8.031905e+04,5.410333,9583.157556,11.006606,0.139725,5.867489,632.608756,0.222222
std,6.045108,8.042250e+04,6.063532,6314.886691,2.978808,0.087212,3.879702,50.435865,0.415744
min,20.000000,8.000000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,390.000000,0.000000
25%,24.000000,4.720400e+04,1.000000,5000.000000,8.590000,0.070000,3.000000,601.000000,0.000000
50%,26.000000,6.704800e+04,4.000000,8000.000000,11.010000,0.120000,4.000000,640.000000,0.000000
75%,30.000000,9.578925e+04,8.000000,12237.250000,12.990000,0.190000,8.000000,670.000000,0.000000
max,144.000000,7.200766e+06,125.000000,35000.000000,20.000000,0.660000,30.000000,850.000000,1.000000


## Outlier Investigation

Investigating suspicious values before deciding how to handle them.

In [11]:
# Investigate age outliers
print("=== PERSON AGE Analysis ===")
print(f"Min: {df['person_age'].min()}")
print(f"Max: {df['person_age'].max()}")
print(f"Mean: {df['person_age'].mean():.1f}")
print(f"\nHow many people are above 100 years old?")
print(f"Count: {len(df[df['person_age'] > 100])}")
print(f"\nAge distribution of people > 60:")
print(df[df['person_age'] > 60]['person_age'].value_counts().sort_index())

=== PERSON AGE Analysis ===
Min: 20
Max: 144
Mean: 27.8

How many people are above 100 years old?
Count: 7

Age distribution of people > 60:
person_age
61     12
62      7
63      3
64      7
65     10
66     11
67      1
69      5
70      7
73      3
76      1
78      1
80      1
84      1
94      1
109     1
116     1
123     2
144     3
Name: count, dtype: int64


In [15]:
# Investigating Employee Experience Outliers

print("=== EMPLOYMENT EXPERIENCE Analysis ===")
print(f"Min: {df['person_emp_exp'].min()}")
print(f"Max: {df['person_emp_exp'].max()}")
print(f"Mean: {df['person_emp_exp'].mean():.1f}")
print(f"\nHow many people have > 50 years experience?")
print(f"Count: {len(df[df['person_emp_exp'] > 50])}")

=== EMPLOYMENT EXPERIENCE Analysis ===
Min: 0
Max: 125
Mean: 5.4

How many people have > 50 years experience?
Count: 12


In [12]:
# Investigating Income Outliers

print("=== PERSON INCOME Analysis ===")
print(f"Min: ${df['person_income'].min():,}")
print(f"Max: ${df['person_income'].max():,}")
print(f"Mean: ${df['person_income'].mean():,.0f}")
print(f"Median: ${df['person_income'].median():,.0f}")
print(f"\nHow many people earn > $500,000?")
print(f"Count: {len(df[df['person_income'] > 500000])}")
print(f"\nHow many people earn > $300,000?")
print(f"Count: {len(df[df['person_income'] > 300000])}")

=== PERSON INCOME Analysis ===
Min: $8,000
Max: $7,200,766
Mean: $80,319
Median: $67,048

How many people earn > $500,000?
Count: 104

How many people earn > $300,000?
Count: 364


## 5. Preprocessing Decisions
Based on our investigation, we will apply the following filters:
| Column | Issue | Decision | Justification |
|--------|-------|----------|---------------|
| `person_age` | Max = 144 (impossible) | Cap at 70 | Reasonable max age for loan applicants |
| `person_emp_exp` | Max = 125 (impossible) | Cap at 50 | Max working career ~50 years |
| `person_income` | Max = $7.2M (extreme outlier) | Cap at $300,000 | Focus on typical borrowers, not millionaires |
**Approach:** We will FILTER OUT rows that exceed these limits, not modify the values.

# 6. Applying Preprocessing Filters

In [13]:
# Storing original row count for comparison
original_rows = len(df)
print(f'Original_dataset: {original_rows} rows')

# Applying filters based on decisions taken
df_clean = df[
    (df['person_age'] <= 70) &
    (df['person_emp_exp'] <= 50 ) &
    (df['person_income'] <= 300000)
]

# Checking how many rows remain
clean_rows = len(df_clean)
removed_rows = original_rows - clean_rows

print(f'After Filtering: {clean_rows} rows')
print(f'Removed: {removed_rows} rows ({removed_rows/original_rows*100:.2f} %)')

Original_dataset: 45000 rows
After Filtering: 44625 rows
Removed: 375 rows (0.83 %)


In [14]:
# Verifying Target variable interpretation
# if previous_loan_defaults = 'yes', expect higher default rates

print("=== Target Variable Verification ===")
print("\nDefault rate by previous loan defaults:")
print(df_clean.groupby('previous_loan_defaults_on_file')['loan_status'].mean())

print("\n\nDefault rate by credit score bands:")
df_clean['credit_band'] = pd.cut(df_clean['credit_score'],
                                 bins = [0, 580, 670,740,850],
                                 labels = ['Poor', 'Fair', 'Good','Excellent'])
print(df_clean.groupby('credit_band')['loan_status'].mean())

=== Target Variable Verification ===

Default rate by previous loan defaults:
previous_loan_defaults_on_file
No     0.453329
Yes    0.000000
Name: loan_status, dtype: float64


Default rate by credit score bands:
credit_band
Poor         0.223978
Fair         0.225174
Good         0.217748
Excellent    0.233766
Name: loan_status, dtype: float64


In [15]:
df_clean = df_clean.drop(columns=['credit_band'])
print('Temporary Column Removed')
print(f'Current Columns: {df_clean.columns.tolist()}')

Temporary Column Removed
Current Columns: ['person_age', 'person_gender', 'person_education', 'person_income', 'person_emp_exp', 'person_home_ownership', 'loan_amnt', 'loan_intent', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length', 'credit_score', 'previous_loan_defaults_on_file', 'loan_status']


# 7. Feature Engineering - Derived Columns

Creating new columns that will be useful for risk analysis.

In [20]:
# 1. Age Groups - Useful for demographic analysis

df_clean['age_group'] = pd.cut(
    df_clean['person_age'],
    bins=[0, 25, 35, 45, 55, 70],
    labels=['18-25', '26-35', '36-45', '46-55', '56-70']
)

# 2. Income Brackets - Useful for financial segmentation
df_clean['income_bracket'] = pd.cut(
    df_clean['person_income'],
    bins=[0, 30000, 60000, 100000, 300000],
    labels=['Low (<30K)', 'Medium (30-60K)', 'High (60-100K)', 'Very High (>100K)']
)

# 3. Credit Score Category - Industry standard bands
df_clean['credit_category'] = pd.cut(
    df_clean['credit_score'],
    bins=[0, 580, 670, 740, 850],
    labels=['Poor', 'Fair', 'Good', 'Excellent']
)

# Check the new columns
print("New columns created!")
print('\n')
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', None)
print(df_clean[['person_age', 'age_group', 'person_income', 'income_bracket', 'credit_score', 'credit_category']].head(10))

New columns created!


   person_age age_group  person_income     income_bracket  credit_score credit_category
0          22     18-25          71948     High (60-100K)           561            Poor
1          21     18-25          12282         Low (<30K)           504            Poor
2          25     18-25          12438         Low (<30K)           635            Fair
3          23     18-25          79753     High (60-100K)           675            Good
4          24     18-25          66135     High (60-100K)           586            Fair
5          21     18-25          12951         Low (<30K)           532            Poor
6          26     26-35          93471     High (60-100K)           701            Good
7          24     18-25          95550     High (60-100K)           585            Fair
8          24     18-25         100684  Very High (>100K)           544            Poor
9          21     18-25          12739         Low (<30K)           640            Fair


In [22]:
print(f'Total Columns: {len(df_clean.columns)}')
print('\nTotal Columns:')
for i, col in enumerate(df_clean.columns,1):
    print(f'{i}. {col}')

Total Columns: 17

Total Columns:
1. person_age
2. person_gender
3. person_education
4. person_income
5. person_emp_exp
6. person_home_ownership
7. loan_amnt
8. loan_intent
9. loan_int_rate
10. loan_percent_income
11. cb_person_cred_hist_length
12. credit_score
13. previous_loan_defaults_on_file
14. loan_status
15. age_group
16. income_bracket
17. credit_category


## 8. Exporting Clean Dataset

Saving the preprocessed dataset for SQL and Power BI analysis.

In [23]:
output_path = '../data/processed/loan_data_cleaned.csv'
df_clean.to_csv(output_path, index = False)

print(f'Cleaned Data exported to : {output_path}')
print(f'Rows: {len(df_clean)}')
print(f'Columns: {len(df_clean.columns)}')

Cleaned Data exported to : ../data/processed/loan_data_cleaned.csv
Rows: 44625
Columns: 17


## 9. Preprocessing Summary
| Metric | Value |
|--------|-------|
| **Original rows** | 45,000 |
| **Final rows** | 44,625 |
| **Rows removed** | 375 (0.83%) |
| **Original columns** | 14 |
| **Final columns** | 17 |
### Filters Applied:
- `person_age` ≤ 70
- `person_emp_exp` ≤ 50
- `person_income` ≤ $300,000
### Derived Columns Added:
1. `age_group` — Age brackets (18-25, 26-35, etc.)
2. `income_bracket` — Income levels (Low, Medium, High, Very High)
3. `credit_category` — Credit score bands (Poor, Fair, Good, Excellent)
### Target Variable:
- `loan_status = 0` → Defaulted
- `loan_status = 1` → Repaid successfully
### Output:
- Cleaned data saved to: `data/processed/loan_data_cleaned.csv`